In [1]:
import gurobipy as gp
from gurobipy import GRB
from eventlet.timeout import Timeout

# import auxillary packages
import requests  # for loading the example source code
import openai
import json

# import flaml and autogen
from flaml import autogen
from flaml.autogen.agentchat import Agent, UserProxyAgent
from flaml.autogen.code_utils import extract_code
from create_bench_agent import CreateBenchAgent



In [2]:
example_qa = """
----------
Param Update: ```JSON
"end_charge_time": 9```
Example User request: Charge the car till 9 AM.
----------
Param Update: ```JSON
"end_charge": 1.00```
Example Request: Charge the car to full charge.
----------
Param Update: ```JSON
"end_charge": 1.00```
Example Request: Charge the car to full charge.
"""

In [3]:
params_filepath = "/Users/viditjain/Desktop/LLM Optimizer/EnergySaverLLM/EnergySaverLLM/Model/params/EVCharging.json"

In [4]:
log_history = {}

In [2]:
autogen.oai.ChatCompletion.start_logging(log_history)

NameError: name 'autogen' is not defined

In [6]:
config_list = autogen.config_list_from_json(
    env_or_file = "OAI_CONFIG_LIST",
    filter_dict={
        "model": [ "gpt-4","gpt-3.5-turbo-16k"],
    },
)

In [7]:
agent = CreateBenchAgent(
    name="CreateBenchAgent",
    example_qa=example_qa,
    json_filepath=params_filepath,
    llm_config={
        "request_timeout": 600,
        "seed": 42,
        "config_list": config_list,
    }
)

user = UserProxyAgent(
    "user", max_consecutive_auto_reply=0,
    human_input_mode="NEVER", code_execution_config=False
)

In [8]:
with open('benchmark_values.json', 'r') as f:
    # text = f.read()
    values_db = json.loads(f.read())

In [9]:
param_update = """```JSON
"{key}": {new_val}```"""

In [10]:
benchmark_dataset = []
for key, val_list in values_db.items():
    for new_val in val_list:
        benchmark_instance = {}
        benchmark_instance["json_str"] = param_update.format(key = key, new_val = new_val)
        agent = CreateBenchAgent(
            name="CreateBenchAgent",
            example_qa=example_qa,
            json_filepath=params_filepath,
            llm_config={
                "request_timeout": 600,
                "seed": 42,
                "config_list": config_list,
            }
        )
        user = UserProxyAgent(
            "user", max_consecutive_auto_reply=0,
            human_input_mode="NEVER", code_execution_config=False
        )
        
        user.initiate_chat(agent, message = benchmark_instance["json_str"], silent = True)

        log_list = []
        log_list = eval(list(log_history.keys())[-1])
        benchmark_instance['prompt'] = ""+(log_list[-1]['content'])

        benchmark_dataset.append(benchmark_instance)

CreateBenchAgent (to writer):




Describe the change:



--------------------------------------------------------------------------------
writer (to CreateBenchAgent):

Change the carbon cost weight to 0.2.

--------------------------------------------------------------------------------
CreateBenchAgent (to writer):




Describe the change:



--------------------------------------------------------------------------------
writer (to CreateBenchAgent):

Change the parameter "carbon_cost_weight" to 0.4.

--------------------------------------------------------------------------------
CreateBenchAgent (to writer):




Describe the change:



--------------------------------------------------------------------------------
writer (to CreateBenchAgent):

Change the maximum allowed electricity cost in one charge cycle to 10 USD.

--------------------------------------------------------------------------------
CreateBenchAgent (to writer):




Describe the change:



-----------------------

In [12]:
for entry in benchmark_dataset:
    print(entry['prompt'])

Change the carbon cost weight to 0.2.
Change the parameter "carbon_cost_weight" to 0.4.
Change the maximum allowed electricity cost in one charge cycle to 10 USD.
I updated the maximum energy cost parameter to 15.
Update the maximum energy cost allowed in one charge cycle to 20 USD.
Set the maximum electricity cost allowed in one charge cycle to 25 USD.
Change "max_energy_cost" parameter to 30.
Update the maximum energy cost allowed in one charge cycle to 35 USD.
Change the maximum energy cost allowed in one charge cycle to 40 USD.
Change the maximum energy cost allowed in one charge cycle to 45 USD.
Charge the car to 60% of its capacity.
Charge the car until 7 AM.
Change the parameter "end_charge" to a value of 0.9.
Charge the car to full charge.
Change the maximum charge power to 10 kW.
Change the maximum charge power to 11.
Change the maximum charge power to 12 KW.
Update the battery capacity parameter to 50 kWh.
Battery capacity has been changed to 60 kWh.
Change the battery capaci

In [13]:
benchmark_dataset_filename = "EV.benchmark.json"

In [14]:
with open(benchmark_dataset_filename, "w") as text_file:
    text_file.write(json.dumps(benchmark_dataset, indent=2))